# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet)).

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv ` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [78]:
# importando bibliotecas
import pandas as pd
import numpy as np
import requests

# 1, i) Importando a base 'SINASC_RO_2019.csv'
sinasc = pd.read_csv(r'F:\Computação\2. EBAC\4. Python em Ciência de Dados\2. Limpeza e preparação de dados\Tarefa 1\SINASC_RO_2019.csv')
print(sinasc.shape)
sinasc.drop_duplicates().shape

(27028, 69)


(27028, 69)

Podemos constatar que não há duplicados nessa base.

In [79]:
# 1, ii) Outra forma de extrair a quantidade de linhas duplicadas, se houver: 

qt_duplicados = len(sinasc.drop_duplicates())

if qt_duplicados == (len(sinasc)):
    print('Não há duplicados!')
else:
    print(f'Há {qt_duplicados} valores duplicados')

Não há duplicados!


In [80]:
# 2) Exibindo os valores NaN pra todas as colunas do df:

sinasc.isna().sum().sort_values(ascending=False)

DTRECORIGA      27028
CODANOMAL       26814
IDADEPAI        19421
SERIESCMAE      12710
DTULTMENST      10072
                ...  
TPROBSON            0
munResUf            0
munResNome          0
munResTipo          0
munResStatus        0
Length: 69, dtype: int64

In [81]:
# 3) Criando uma pd.Series com os valores faltantes apenas com as colunas mais relevantes para a análise:

# i) Criando uma lista com as colunas que nos são relevantes
lista_colunas = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
                 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 

# ii) Criando um novo DataFrame filtrado apenas com as colunas que nos são relevantes.
sinasc_filtrado = sinasc.loc[: ,lista_colunas].copy()

# iii) Contando a quantidade de nulos no novo DataFrame:
quantidade_nulos = sinasc_filtrado.loc[: , :].isna().sum().sort_values(ascending=False)
quantidade_nulos

QTDFILVIVO    1573
GESTACAO      1232
ESTCIVMAE      317
ESCMAE         312
APGAR5         103
GRAVIDEZ        79
LOCNASC          0
IDADEMAE         0
CONSULTAS        0
dtype: int64

In [82]:
# 4, i) Tratamento para Apgar5: removendo valores NaN

sinasc_filtrado = sinasc_filtrado.dropna(subset=['APGAR5'])
sinasc_filtrado

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [83]:
# 4, ii) Contando número de linhas e o número de missing_values:

print(sinasc_filtrado.shape)
sinasc_filtrado.drop_duplicates().shape

(26925, 9)


(9956, 9)

In [84]:
# 5) Determinando missing_values nas colunas ESTCIVMAE e CONSULTAS para ser == 9

# i) Usando o método .fillna para substituir os NaN por 9.
sinasc_filtrado[['ESTCIVMAE', 'CONSULTAS']] = sinasc_filtrado[['ESTCIVMAE', 'CONSULTAS']].fillna(9)
sinasc_filtrado

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [85]:
sinasc_filtrado.columns

Index(['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO',
       'GRAVIDEZ', 'CONSULTAS', 'APGAR5'],
      dtype='object')

In [86]:
# 6) Substituindo valores faltantes na coluna QTDFILVIVO para 0:

# i) Usando o método .fillna para substituir os valores nulos em ['QTDFILVIVO'] para 0.
sinasc_filtrado['QTDFILVIVO'] = sinasc_filtrado['QTDFILVIVO'].fillna(0)
sinasc_filtrado

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


In [87]:
# 7) Decidir o processo de limpeza dos datos faltantes para as colunas restantes.

# 7, i) Calcular as colunas que ainda tem valores nulos:
sinasc_filtrado.isna().any().sort_values(ascending=False)

ESCMAE         True
GESTACAO       True
GRAVIDEZ       True
LOCNASC       False
IDADEMAE      False
ESTCIVMAE     False
QTDFILVIVO    False
CONSULTAS     False
APGAR5        False
dtype: bool

Agora, sabemos que as colunas `ESCMAE`, `GESTACAO`, e `GRAVIDEZ` ainda possuem valores nulos. Vamos extrair o texto de cada uma delas do arquivo 'ESTRUTURA_SINASC' para decidirmos o que fazer com os nulos dessas colunas:

| Coluna    | Descrição                                       | Valores                                                          |
|-----------|-------------------------------------------------|------------------------------------------------------------------|
| ESCMAE    | Escolaridade, anos de estudo concluídos         | 1: Nenhuma<br>2: 1 a 3 anos<br>3: 4 a 7 anos<br>4: 8 a 11 anos<br>5: 12 e mais<br>9: Ignorado |
| GESTACAO  | Semanas de gestação, conforme a tabela          | 9: Ignorado<br>1: Menos de 22 semanas<br>2: 22 a 27 semanas<br>3: 28 a 31 semanas<br>4: 32 a 36 semanas<br>5: 37 a 41 semanas<br>6: 42 semanas e mais |
| GRAVIDEZ  | Tipo de gravidez, conforme a tabela             | 9: Ignorado<br>1: Única<br>2: Dupla<br>3: Tripla e mais          |


Com base nesses dados de cada coluna da tabela, podemos tomar algumas decisões acerca do tratamento dos dados. Por exemplo, vamos analisar a distribuição básica de estatísticas de cada coluna de interesse, assim, podemos estabelecer de forma mais eficaz como lidaremos com os valores ausentes.

In [88]:
# 7, ii) Checando a os tipos de cada variável de interesse

sinasc_filtrado.dtypes

LOCNASC         int64
IDADEMAE        int64
ESTCIVMAE     float64
ESCMAE         object
QTDFILVIVO    float64
GESTACAO       object
GRAVIDEZ       object
CONSULTAS       int64
APGAR5        float64
dtype: object

In [89]:
# 7, iii) Averiguando a estatística descritiva das variáveis de interesse:

sinasc_filtrado.select_dtypes('object').describe()

,ESCMAE,GESTACAO,GRAVIDEZ
count,26615,25709,26849
unique,5,6,3
top,8 a 11 anos,37 a 41 semanas,Única
freq,15581,22523,26356


In [90]:
# Estatisticas básicas de cada top valor nas colunas de interesse:

gravidez_unica = (26356 / 26849 ) * 100
gestacao_37_41_semanas = (22523 / 25709) * 100
escmae_8_11_anos = (15581 / 26615) * 100

print(gravidez_unica, gestacao_37_41_semanas, escmae_8_11_anos)


98.16380498342583 87.60745264304329 58.542175464963364


Agora que temos os dados descritivos de cada coluna, podemos tomar algumas decisões com os dados. Por exemplo:
*   A coluna `GRAVIDEZ` possui 26.356 valores iguais a 'Única', o que representa **98.1% dos dados**. Isso significa que valores diferentes de 'Única' são muito raros, e por isso, assumiremos que os valores nulos tem altíssima chance de também serem 'Única', e converteremos os dados dos valores nulos para 'Única'.
*   A coluna `GESTACAO` possui também uma quantidade grande de valores iguais a uma única categoria, onde 87% dos valores são iguais a `37 a 41 semanas	`. Para esses casos, convertemos os valores nulos em `9. Ignorado`, para distinguir os dados não completados.
*   Já a coluna `ESCMAE` tem uma distribuição um pouco menos brusca. Podemos investigar mais a fundo a distribuição de cada valor em ESCMAE:

In [91]:
# Usando o parâmetro 'normalize=True' de value_counts para exibir a frequência de cada categoria em 'ESCMAE', e calculando o percentual de valores nulos em ESCMAE
frequencias_escmae = sinasc_filtrado['ESCMAE'].value_counts(normalize=True) * 100

# Calculando a frequência dos valores nulos
frequencia_nulos_escmae = ((sinasc_filtrado['ESCMAE'].isna().sum()) / (sinasc_filtrado['ESCMAE'].value_counts().sum()) * 100).round(2)

# Exibindo os resultados
print(frequencias_escmae)
print()
print(f'A frequência de valores nulos na coluna é {frequencia_nulos_escmae}%.')

ESCMAE
8 a 11 anos        58.542175
12 anos ou mais    21.630659
4 a 7 anos         17.760661
1 a 3 anos          1.916213
Nenhuma             0.150291
Name: proportion, dtype: float64

A frequência de valores nulos na coluna é 1.16%.


Como podemos ver, os dados de 'ESCMAE' possuem poucos valores nulos e não afetam a saúde da coluna, e assim, iremos atribuir o valor de `9. Ignorado` para esses valores, distinguindo claramente os dados não preenchidos para evitar suposições errôneas.

Vamos aplicar as formas de tratamentos de nulos que discutimos para cada coluna:

In [92]:
# 7, iv) Aplicando o tratamento dos dados:

# i) Convertendo NaN de 'GRAVIDEZ' para 'Única':
sinasc_filtrado['GRAVIDEZ'] = sinasc_filtrado['GRAVIDEZ'].fillna('Única')

# ii) Convertendo NaN de 'GESTACAO' para 'Ignorado'
sinasc_filtrado['GESTACAO'] = sinasc_filtrado['GESTACAO'].fillna('Ignorado')

# iii) Convertendo NaN de 'ESCMAE' também para 'Ignorado':
sinasc_filtrado['ESCMAE'] = sinasc_filtrado['ESCMAE'].fillna('Ignorado')

sinasc_filtrado

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0
...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0


8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  


In [93]:
# 8) Categorizando os dados numéricos de 'APGAR5' para diferentes faixas de asfixia nos primeiros 5 minutos:

# usamos a função o método .loc para localizar um DataFrame filtrado e adicionamos esse valor numa nova coluna 'nivel_asfixia' com as respectivas faixas de asfixia:
sinasc_filtrado.loc[sinasc_filtrado['APGAR5'] <= 3, 'nivel_asfixia'] = 'Asfixia Severa'
sinasc_filtrado.loc[(sinasc_filtrado['APGAR5'] >= 4) & (sinasc_filtrado['APGAR5'] <= 5), 'nivel_asfixia'] = 'Asfixia Moderada'
sinasc_filtrado.loc[(sinasc_filtrado['APGAR5'] >= 6) & (sinasc_filtrado['APGAR5'] <= 7), 'nivel_asfixia'] = 'Asfixia Leve'
sinasc_filtrado.loc[sinasc_filtrado['APGAR5'] >= 8, 'nivel_asfixia'] = 'Normal'

sinasc_filtrado


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,nivel_asfixia
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,Normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,Normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,Normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,Normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,Normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,Normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,Normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,Normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,Normal


In [94]:
# 9) Tornando todas as colunas em snake_case:

sinasc_filtrado.columns = sinasc_filtrado.columns.str.lower().str.replace(' ', '_')
sinasc_filtrado

,locnasc,idademae,estcivmae,escmae,qtdfilvivo,gestacao,gravidez,consultas,apgar5,nivel_asfixia
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,Normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,Normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,Normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,Normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,Normal
...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,Normal
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,Normal
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,Normal
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,Normal


### Recapitulando...
Neste exercício, aprendi bastante sobre tratamento e manipulação de dados ausentes no Pandas, especialmente sobre métodos de tratamento de dados ausentes, formas de calcular e exibir estatísticas de dados ausentes, e a representação dos dados ausentes no dataset. Com a ajuda de composições de funções e métodos nativos do Python e da biblioteca Pandas, como .isna().sum() para retornar uma soma de valores nulos, ou .value_counts().sum() para somar a quantidade de cada valor de uma coluna, nós podemos criar uma análise robusta e poderosa para tratar e limpar dados no Python.